<a href="https://colab.research.google.com/github/vascomedici/teaching_project_scc/blob/main/pv_simulator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. PV plant simulator
This first example is a **simple PV plant simulator**, which simulates the output of an unshaded PV plant, starting from a location and a weather file that contains T and GHI. We will use the [pvlib library](https://pvlib-python.readthedocs.io/) to help us with some calculations. This is a very powerful PV simulation library that can be used to simulate PV plants in detail. To keep it simple, here we don't use it to its full capabilities.

First, we load the **weather csv** file into a pandas dataframe

In [ ]:
%%capture
!pip install pvlib
!gdown --id 1Psvmk5z3zlB8YBmK5PR0vd4J0RLirgKG # download meteo data from shared gdrive file
import pvlib
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt

date_parser = lambda x: datetime.strptime(x, '%d-%b-%Y %H:%M:%S')
meteo = pd.read_csv('meteo.csv', header=0, parse_dates=[0], date_parser=date_parser, index_col=0)

Let's plot it

In [ ]:
meteo.plot(subplots=True);

We define the plant location and orientation

In [ ]:
# location
latitude = 47.03
longitude = 8.296
altitude = 460
location = pvlib.location.Location(latitude, longitude, 'Etc/GMT+1', altitude)

# PV plant
p_module = 300  # potenza del modulo in 300W (1m x 1.7m) (non cambiare)
num_modules = 66  # numero di moduli
tilt = 15
azimuth = 193
albedo = 0.1 # albedo of the ground for ground reflected irradiance

We now calculate the irradiance on the plane of array in the following way:
*   We calculate the sun position based on location and time
*   We split GHI into its direct and diffuse components
*   And project them on the plane of array (POA)

In [ ]:
sun_pos = location.get_solarposition(meteo.index)
ghi = meteo['GHI']
dni = pvlib.irradiance.dirint(meteo['GHI'], sun_pos['zenith'], sun_pos.index)
dni.fillna(0, inplace=True)
dhi = ghi - np.cos(np.deg2rad(sun_pos['zenith'])) * dni


irr = pvlib.irradiance.get_total_irradiance(tilt, azimuth, sun_pos['zenith'], sun_pos['azimuth'], dni, ghi, dhi,
                                      albedo=albedo, model_perez='allsitescomposite1990')

We calculate the power of the PV plant with a simplified model that takes into account temperature and inverter efficiency.
We calculate the cell temperature using the [Sandia model](https://pvlib-python.readthedocs.io/en/stable/generated/pvlib.temperature.sapm_cell.html), which is based on these two empirical equations:

$$T_{m} = E \cdot \exp (a + b \cdot WS) + T_{a}$$
$$T_{C} = T_{m} + \frac{E}{E_{0}} \Delta T$$

We then correct for module and inverter efficiency as a function of GHI using the following formula:

$$ \eta = k2 + k3 \cdot \log(GHI / GHI_{STC}) +k4 \cdot\log(GHI/GHI_{STC})^2 $$


In [ ]:
# correction for angle of incidence using IAM model
b0 = 0.05
aoi = pvlib.irradiance.aoi(tilt, azimuth, sun_pos['zenith'], sun_pos['azimuth'])  # angle of incidence
iam = np.maximum(1 - b0 * (1 / np.cos(np.minimum(np.deg2rad(aoi), np.pi/2)) - 1), 0)
irr['poa_eff'] = iam * irr['poa_direct'] + 0.95 * (irr['poa_diffuse'])

# correction for temperature
gamma = -0.34 / 100  # power loss as function of temperature
t_nom = 25  # nominal temperature
a=-3.56 #
b=-0.075
deltaT=3
wind_speed=1
i_stc = 1000

t_cell = pvlib.temperature.sapm_cell(irr['poa_global'], meteo['Ta'], wind_speed, a, b, deltaT, irrad_ref=i_stc)
irr['poa_eff_tc'] = irr['poa_eff'] * (1 + gamma * (t_bom - t_nom))

# correction for low irrandiance power losses of module and inverter
k2 = 0.942
k3 = -5.02e-2
k4 = -3.77e-2
np.seterr(divide = 'ignore') # disable divide by zero warning 
eta = np.maximum(0.1, (k2 + k3 * np.log(irr['poa_eff'] / i_stc) + k4 * np.log(irr['poa_eff'] / i_stc)**2).fillna(0))
np.seterr(divide = 'warn') 
irr['poa_eff_tc_li'] = irr['poa_eff_tc'] * eta

We calculate the power of the PV plant

In [ ]:
# power of plant in W
p_plant = p_module * irr['poa_eff_tc_li'] / 1000 * num_modules
p_plant.plot()

# calculate the energy of the plant in kWh
e_plant = p_plant / 1000 * (p_plant.index[1]- p_plant.index[0]).seconds / 3600 # W to kWh

# here this is useless
save_path = 'energy_pv.csv'
e_plant.to_csv(save_path, sep=',', date_format='%Y-%m-%dT%H:%M:%S', float_format='%.03f')

# this downloads the csv file onto your local machine
# from google.colab import files
# files.download('energy_pv.csv')

# 2. Consumption data exploration
In the following cells, we dowload the consumption data from the aparments into a pandas dataframe and plot them

In [ ]:
!gdown --id 1F8rP6NoZiTUsu9nNEMN3suIhStaSCn78
date_parser = lambda x: datetime.strptime(x, '%d/%m/%Y %H:%M')
consumption = pd.read_csv('dati_consumo_15min.csv', header=0, parse_dates=[0], date_parser=date_parser, index_col=0)
consumption.head()

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(30,5))
E15 = consumption.sum(axis=1)
E15.plot(ax=ax[0])
ax[0].set_title('full year')
ax[0].set_xlabel('total consumption [kWh]')
E15.iloc[consumption.index<'2019-01-07 00:15:00'].plot(ax=ax[1])
ax[0].set_title('full year')
ax[0].set_xlabel('total consumption [kWh]')
ax[1].set_title('first 7 days')
ax[1].set_xlabel('total consumption [kWh]')

# 3. Return on investment estimate for PV plant
We estimate the return on investment of a PV plant, given:

*   Production profile (calculated above)
*   Cost of the PV plant
*   Operation and maintenance costs
*   Energy prices
*   Self-consumption
*   Cost of capital
*   Expected life of the PV plant

We first install the necessary libraries and define the parameters

In [ ]:
%%capture
!pip install numpy_financial
import numpy_financial as npf  # used to calculate financial figures
o_and_m = 30  # operation and maintenace costs [CHF/kW]
wacc = 0.02  # weighted average cost of capital
life_span = 25
degradation = 0.2  # 20% after 25years

self_consumption = 0.6  # 60%
p_sell = 0.06
p_buy = 0.21
p_pv = p_buy * self_consumption + p_sell * (1 - self_consumption)

plant_size = p_module * num_modules / 1000
plant_cost = 34520

Let's **calculate the PV production during the 25 years life span and take into account the degratation** of the PV plant.
Total degradation after life span should be 20%, we can calculate the degradation at each time step using the following formulae:

$$(1-d) = (1-d_m)^{mt}$$
$$d_m = 1-(1-d)^{1/mt}$$

where $d$ is the total degradation at the end of the life, $t$ is the life span in years and $m$ is the number of steps per year

In [ ]:
e_ls = np.tile(e_plant, life_span)
d_m = 1 - (1-degradation)**(1/len(e_ls))
eta = (1-d_m)**np.arange(len(e_ls))
e_ls *= eta

plt.figure(figsize=(20,12))
plt.plot(e_ls);

Now we perform a very simplistic financial analysis to estimate the return on investment.
First, we calculate all costs and revenues. Then we calculate the free cash flow for the life of the PV system and then calculate the net present value for each year to calculate after how many years we reach a break-even condition. In this very simple analysis, we don't take taxes into account.

##NPV
In economics, the net present value (abbreviated as NPV) is a method used to define the present value of an expected series of cash flows not only by adding them up in the accounts but by discounting them on the basis of the rate of return.
$$\mathrm{NPV}(i, N) = \sum_{t=0}^N \frac{R_t}{(1+i)^t}$$

##IRR
The Internal Rate of Return (IRR) is the rate of the exponential law that makes a financial asset fair.

$$\mathrm{NPV}(IRR, N) = 0$$

In [ ]:
years = range(life_span)
capex = plant_cost
opex = np.ones(life_span) * o_and_m * plant_size
costi = np.hstack([capex, opex])

yearly_production = np.sum(np.reshape(e_ls, (life_span, -1)), axis=1)
revenues = yearly_production * p_pv

fcf = np.hstack([0, revenues]) - costi  # free cash flow
npv = np.array([npf.npv(wacc, fcf[:i]) for i in np.arange(life_span + 1)])
irr = npf.irr(fcf)

financials = pd.DataFrame(data={'years': np.arange(26), 'NPV': npv, 'FCF': fcf}).set_index('years')

fig, ax = plt.subplots(1,1,figsize=(20,12))

financials.plot(kind='bar', title='IRR at {} years: {:.2%}'.format(life_span, irr), ax=ax).set_ylabel('CHF')